In [5]:
!pip install alphabets

In [ ]:
import os.path # To handle paths

In [6]:
en_alphabet = list(range(97, 123))
 es_alphabet = en_alphabet.copy()
es_alphabet.append(241)

all_ascii = list(range(256))
non_alphabetic_en = [simbol for simbol in all_ascii if simbol not in en_alphabet] 
non_alphabetic_es = [simbol for simbol in all_ascii if simbol not in es_alphabet]

alphabets = {
    "en_alphabet": en_alphabet,
    "es_alphabet": es_alphabet,
    "non_alphabetic_en": non_alphabetic_en,
    "non_alphabetic_es": non_alphabetic_es
 # We create our alphabets separating english from spanish and there respective non alphabetical charaters }

In [8]:
def scan_model(text_file:str, language="en")->dict:

    """
    Reads 'text_file' in 'language' format, clears nonalphabetic symbols and counts frequencies
    """
    
    with open(text_file, 'r', encoding='utf-8') as file:
        text = (file.readlines())
    all_words = "".join(text)
            
    hist = dict()

    if len(all_words) != 0:
        all_words = all_words.lower()
        for code in alphabets[f"non_alphabetic_{language}"]:
            all_words = all_words.replace(chr(code), "")

        for code in alphabets[f"{language}_alphabet"]:
            hist[chr(code)] = all_words.count(chr(code))/len(all_words)
    
    return hist

en_model = scan_model(os.path.join("Train_data","en_train.txt"), "en") 
es_model = scan_model(os.path.join("Train_data","es_train.txt"), "es") 
# 2 models are now trained by reading texts, the english model will read harry potter and will count the amount of letter 
creating the frequencies of these(non alphabetical chars are removed), the same will be done in spanish with El quijote de la mancha 


In [ ]:
from math import log

def scan_test(text_file:str, language="en")->str:

    """
    Reads 'text_file' in 'language' format, clears nonalphabetic symbols
    """

    with open(text_file, 'r', encoding='utf-8') as file:
        text = (file.readlines())
    all_words = "".join(text)
    all_words = all_words.lower()
    for code in alphabets[f"non_alphabetic_{language}"]:
        all_words = all_words.replace(chr(code), "")

    return all_words

def likelihood(string:str, probs:dict):
    result = 1.0
    for letter in string:
        result = result*probs[letter]

    return result

def log_likelihood(string:str, probs:dict):
    result = 0.0
    for letter in string:
        if probs[letter] != 0:
            result = result + log(probs[letter])
    return result

In [ ]:
Text = str.lower(input ("Ingrese una frase en español o ingles: "))
Text = Text.replace(" ","")
spa_eng = int(input ("Si escribio en ingles marque 1 si fue en español escriba 0: "))

english = log_likelihood(Text, en_model), likelihood(Text, en_model)
spanish = log_likelihood(Text, es_model), likelihood(Text, es_model)


if english[1]>spanish[1]:
  print("Predigo que es Ingles")
  if spa_eng == 1:
    print("Se predijo correctamente con ")
  else :
    print("No pude predecir bien")
else:
    print("Predigo que es Español")
    if spa_eng == 0:
      print("Se predijo correctamente")
    else :
      print("No pude predecir bien")


NameError: name 'log_likelihood' is not defined